In [1]:
"""!pip install googledrivedownloader
!pip install keras"""

In [2]:
"""from google_drive_downloader import GoogleDriveDownloader as gdd

ids =["1O_5TNdgJUO7P5CBX_uDUjIFstSLCEYSg","1lT76n-lH3znZ7KIql1cEqGaMH-m-v1oS", 
     "1vWqNRs0kMDNSj7kmjAQZUhoKX9Xi-RhY"]

file_names = ["glove.6B.100d.txt","train.csv","test.csv"]
for i in range(len(ids)):
  a = gdd.download_file_from_google_drive(file_id= ids[i],
                                      dest_path="./"+file_names[i],
                                      unzip=False)"""

In [3]:
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
from keras.models import Model
from keras import backend as K    
K.set_image_dim_ordering('th') 

Using TensorFlow backend.


In [0]:
train_data = pd.read_csv("../data/train.csv", header=0)
test_data = pd.read_csv("../data/test.csv", header=0)

In [32]:
X_train = train_data["Tweet"]
Y_train = train_data["Category"]
print X_train.shape

X_test = test_data["Tweet"]
Y_test = test_data["Category"]
print X_test.shape

(8264,)
(2140,)


In [0]:
#Increasing the value will increase sequence length in many sentences. Captures more words
TOP_N_WORDS = 5000

In [34]:
tokenizer = Tokenizer(nb_words=TOP_N_WORDS)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 25498 unique tokens.


In [35]:
#Tokenizer.word_index shows the word and its index in the dictionary
#These indices are fed as a sequence
print (tokenizer.word_index)

{'jvxycjgtbl': 23274, 'bdfwhotgw9': 19679, 'gavar': 12038, 'woods': 7776, 'hanging': 4349, 'katrinakaif': 12712, '9ykbg7gcfm': 9452, 'jlyeykphny': 17733, 'ourfriends': 23221, 'mutinies': 13461, 'dw10ht5lgv': 18615, 'krattiger': 21949, 'taj': 12627, 'bringing': 2521, 'advices': 24964, 'wooden': 4599, 'wednesday': 7190, 'yqtelnpfdx': 22660, 'q22snc3s8v': 23671, '\xe2\x9d\xa4\xef\xb8\x8f\xe2\x9d\xa4\xef\xb8\x8f\xe2\x9d\xa4\xef\xb8\x8f\xf0\x9f\x8f\x86': 9482, '273': 22934, 'sh1t': 15318, 'errors': 16049, 'cooking': 6946, 'warmongering': 16522, 'xtrcuedeon': 21628, 'designing': 11649, 'databank': 24931, '920million': 15702, 'shocks': 6827, 'mccutchen': 17956, 'awovxnycd0': 23694, 'wptdy126yz': 13300, '\xf0\x9f\x98\x8e\xf0\x9f\x98\x8ewe': 9360, 'brainwashed': 6974, 'china': 590, 'gcupvytlsj': 14768, 'affiliated': 25449, '\xf0\x9f\x98\xab\xf0\x9f\x98\x8d': 10200, 'kids': 1018, '6maakyvxny': 21175, 'controversy': 11484, 'kidd': 18646, 'golden': 1590, 'projection': 14925, 'ocrg6daf5d': 19153, '

In [36]:
#This will show how the first sentence has been converted to numeric sequence
print sequences[0]
#This will show the first sentence itself
print "Sentence: "+X_train[0]
#This will loop through every word of the first sentence and see which word is not added in the sequence.
print "Comment: Words not added from the first sentence along with their ID`s"
words = X_train[0].split(" ")
for i in words:
    if i in word_index:
        if word_index[i] not in sequences[0]:
            print i, word_index[i]

[16, 53, 778, 4, 450, 191, 6, 21, 48, 52, 6, 4, 11, 4, 21, 1191, 102, 16, 322, 106, 4, 1110, 7, 14, 52, 72, 165, 4, 79, 191, 23, 4, 663, 17, 10, 4, 3517, 5, 3, 1, 2]
Sentence: i just watched the second part of “revolting rhymes”. it was one of the for the .... it didn’t win. i always watch the nominees and this one only had the first part at the . .... yes that is the intro to https://t.co/92hhi3vqub
Comment: Words not added from the first sentence along with their ID`s
“revolting 8531


In [37]:
max_sequence_length = 0
min_sequence_length = -1
j = -1
for i in sequences:
    seq_len = len(i)
    
    if min_sequence_length == -1:
        min_sequence_length = seq_len
        
    if seq_len > max_sequence_length:
        max_sequence_length = seq_len
    
    if seq_len < min_sequence_length and min_sequence_length!=-1:
        min_sequence_length = seq_len
        j = i
        
print min_sequence_length
print max_sequence_length

0
2547


In [0]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [39]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y_train)
encoded_Y = encoder.transform(Y_train)
# convert integers to dummy variables (i.e. one hot encoded)
Y_train = np_utils.to_categorical(encoded_Y)
print Y_train

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [40]:
data = pad_sequences(sequences, maxlen=max_sequence_length)
import numpy as np
data_labels = Y_train
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', data_labels.shape)

('Shape of data tensor:', (8264, 2547))
('Shape of label tensor:', (8264, 4))


In [41]:
print data[0]
print data_labels[0]

[0 0 0 ... 3 1 2]
[1. 0. 0. 0.]


In [42]:
import os
GLOVE_DIR = "/home/manoj/Downloads/glove.6B/"
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print "Loaded "+str(len(embeddings_index))+" word embeddings from GLOVE"

Loaded 400000 word embeddings from GLOVE


In [0]:
EMBEDDING_DIM = len(embeddings_index["the"])

In [0]:
#+1 for bias.
#len(word_index) because we have so many unique tokens after all the filtering.

embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be random numbers.
        embedding_matrix[i] = embedding_vector

In [45]:
embedding_matrix.shape

(25499, 100)

In [0]:
embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=max_sequence_length,
                            trainable=True)

In [0]:
sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Flatten()(x)
preds = Dense(4, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [48]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 2547)              0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 2547, 100)         2549900   
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 2543, 128)         64128     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 508, 128)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 65024)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 260100    
Total params: 2,874,128
Trainable params: 2,874,128
Non-trainable params: 0
_________________________________________________________________


In [49]:
sequences = tokenizer.texts_to_sequences(X_test)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 25498 unique tokens.


In [50]:
encoder = LabelEncoder()
encoder.fit(Y_test)
encoded_Y = encoder.transform(Y_test)
# convert integers to dummy variables (i.e. one hot encoded)
Y_test = np_utils.to_categorical(encoded_Y)
print Y_test

[[1. 0. 0. 0.]
 [1. 0. 0. 0.]
 [1. 0. 0. 0.]
 ...
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]]


In [51]:
data_test = pad_sequences(sequences, maxlen=max_sequence_length)
import numpy as np
data_test_labels = Y_test
print('Shape of data tensor:', data_test.shape)
print('Shape of label tensor:', data_test_labels.shape)

('Shape of data tensor:', (2140, 2547))
('Shape of label tensor:', (2140, 4))


In [52]:
model.fit(data, data_labels,epochs=40, batch_size=128, validation_data= (data_test,data_test_labels))

Train on 8264 samples, validate on 2140 samples
Epoch 1/40
8264/8264 [==============================] - 11s 1ms/step - loss: 1.8319 - acc: 0.3856 - val_loss: 1.2874 - val_acc: 0.4079
Epoch 2/40
8264/8264 [==============================] - 11s 1ms/step - loss: 0.9802 - acc: 0.6052 - val_loss: 1.0689 - val_acc: 0.5364
Epoch 3/40
8264/8264 [==============================] - 11s 1ms/step - loss: 0.7130 - acc: 0.7397 - val_loss: 1.0331 - val_acc: 0.5659
Epoch 4/40
8264/8264 [==============================] - 11s 1ms/step - loss: 0.5884 - acc: 0.7858 - val_loss: 1.0447 - val_acc: 0.5682
Epoch 5/40
1920/8264 [=====>........................] - ETA: 7s - loss: 0.5081 - acc: 0.81828264/8264 [==============================] - 11s 1ms/step - loss: 0.4946 - acc: 0.8185 - val_loss: 1.0726 - val_acc: 0.5687
Epoch 6/40
8264/8264 [==============================] - 11s 1ms/step - loss: 0.4264 - acc: 0.8479 - val_loss: 1.1934 - val_acc: 0.5617
Epoch 7/40
8264/8264 [==============================] - 11s 1